In [2]:
import os
os.environ["ALFRED_ROOT"] = '/root/data/home/hoyeung/alfred/'

import cv2
import matplotlib.pyplot as plt

import json
from scripts.sample_demo_trajectories import sample_task_trajs

import constants
from env.thor_env import ThorEnv
from agents.deterministic_planner_agent import DeterministicPlannerAgent
from game_states.task_game_state_full_knowledge import TaskGameStateFullKnowledge

In [2]:
class args():
    pass

args.force_unsave_successes = False
args.save_failed = False

args.save_path = '/root/data_alfred/demo_generated/'

args.x_display = '0'

args.in_parallel = False
args.num_threads = 0
args.json_file = ''

args.goal = 'random'
args.pickup = 'random'
args.movable = 'random'
args.receptacle = 'random'
args.scene = 'random'
args.seed = -1

args.trials_before_fail = 5

## Setup

In [3]:
# create env and agent
env = ThorEnv()
game_state = TaskGameStateFullKnowledge(env)
agent = DeterministicPlannerAgent(thread_id=0, game_state=game_state)

ThorEnv started.
planned logs_gen/planner/generated_problems/problem_18828.pddl in 1.70742, plan length 9 solver type 4
planned logs_gen/planner/generated_problems/problem_18828.pddl in 1.71150, plan length 10 solver type 5
planned logs_gen/planner/generated_problems/problem_18828.pddl in 1.74590, plan length 9 solver type 3


Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/envs/pytorch-py3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/envs/pytorch-py3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/envs/pytorch-py3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/pytorch-py3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/pytorch-py3.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/opt/conda/envs/pytorch-py3.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/opt/conda/envs/pytorch-py3.6/lib/python3.6/multiproc

In [4]:
goal_to_required_variables = {"pick_and_place_simple": {"pickup", "receptacle", "scene"},
                              "pick_two_obj_and_place": {"pickup", "receptacle", "scene"},
                              "look_at_obj_in_light": {"pickup", "receptacle", "scene"},
                              "pick_clean_then_place_in_recep": {"pickup", "receptacle", "scene"},
                              "pick_heat_then_place_in_recep": {"pickup", "receptacle", "scene"},
                              "pick_cool_then_place_in_recep": {"pickup", "receptacle", "scene"},
                              "pick_and_place_with_movable_recep": {"pickup", "movable", "receptacle", "scene"}}
goal_to_pickup_type = {'pick_heat_then_place_in_recep': 'Heatable',
                       'pick_cool_then_place_in_recep': 'Coolable',
                       'pick_clean_then_place_in_recep': 'Cleanable'}
goal_to_receptacle_type = {'look_at_obj_in_light': "Toggleable"}
goal_to_invalid_receptacle = {'pick_heat_then_place_in_recep': {'Microwave'},
                              'pick_cool_then_place_in_recep': {'Fridge'},
                              'pick_clean_then_place_in_recep': {'SinkBasin'},
                              'pick_two_obj_and_place': {'CoffeeMachine', 'ToiletPaperHanger', 'HandTowelHolder'}}

scene_id_to_objs = {}
obj_to_scene_ids = {}
scenes_for_goal = {g: [] for g in constants.GOALS}
scene_to_type = {}

# objects-to-scene and scene-to-objects database
for scene_type, ids in constants.SCENE_TYPE.items():
    for id in ids:
        obj_json_file = os.path.join('layouts', 'FloorPlan%d-objects.json' % id)
        with open(obj_json_file, 'r') as of:
            scene_objs = json.load(of)

        id_str = str(id)
        scene_id_to_objs[id_str] = scene_objs
        for obj in scene_objs:
            if obj not in obj_to_scene_ids:
                obj_to_scene_ids[obj] = set()
            obj_to_scene_ids[obj].add(id_str)

# scene-goal database
for g in constants.GOALS:
    for st in constants.GOALS_VALID[g]:
        scenes_for_goal[g].extend([str(s) for s in constants.SCENE_TYPE[st]])
    scenes_for_goal[g] = set(scenes_for_goal[g])

# scene-type database
for st in constants.SCENE_TYPE:
    for s in constants.SCENE_TYPE[st]:
        scene_to_type[str(s)] = st

goal_candidates = constants.GOALS[:]

# Union objects that can be placed.
pickup_candidates = list(set().union(*[constants.VAL_RECEPTACLE_OBJECTS[obj] for obj in constants.VAL_RECEPTACLE_OBJECTS]))
pickup_candidates = [p for p in pickup_candidates if constants.OBJ_PARENTS[p] in obj_to_scene_ids]

# objects that can be used as movable receptacle
movable_candidates = list(set(constants.MOVABLE_RECEPTACLES).intersection(obj_to_scene_ids.keys()))

# objects that can be used as receptacle
receptacle_candidates = [obj for obj in constants.VAL_RECEPTACLE_OBJECTS
                         if obj not in constants.MOVABLE_RECEPTACLES and obj in obj_to_scene_ids] + \
                        [obj for obj in constants.VAL_ACTION_OBJECTS["Toggleable"]
                         if obj in obj_to_scene_ids]
# toaster isn't interesting in terms of producing linguistic diversity
receptacle_candidates.remove('Toaster')
receptacle_candidates.sort() 

# scene IDs
scene_candidates = list(scene_id_to_objs.keys())

## custom args

In [5]:
args.debug = True

In [6]:
add_requirements = {'seed': [43] * args.trials_before_fail, 'obj_repeat':4}

In [7]:
task_tuple = ('pick_two_obj_and_place', 'Watch', 'None', 'Dresser', 205)
# task_name = '%s-%s-%s-%s-%d' % (gtype, pickup_obj, movable_obj, receptacle_obj, scene_num)

## call sampler with fully defined task tuple

In [ ]:
sample_task_trajs(args, task_tuple, agent, env, obj_to_scene_ids, scene_id_to_objs, pickup_candidates, add_requirements={})

Force Unsave Success Data: False
Task: ('pick_two_obj_and_place', 'Watch', 'None', 'Dresser', 205)
PDDLGoalType: pick_two_obj_and_place
Saving images to: /root/data_alfred/demo_generated/new_trajectories/pick_two_obj_and_place-Watch-None-Dresser-205/trial_T20200611_231320_228635/raw_images/
Resetting ThorEnv
set seed in game_state_base reset 125600905
setup random goal ----------------------------------------------
seed None
info ('train', 474)
--------------------------------------------------------------------


Type: train Row:  18828 Scene FloorPlan205 seed 125600905
problem id 18828
Task: put two dresser in 
Performing reset via thor_env API
> /root/data/home/hoyeung/alfred/gen/scripts/sample_planner_trajectories.py(278)sample_task_trajs()
-> env.reset(scene_num)
(Pdb) type(scene_num)
<class 'int'>
(Pdb) c
Resetting ThorEnv
Performing restore via thor_env API
plan
000: GotoLocation loc: loc|-6|9|2|45
001: PickupObject Watch|-01.45|+00.76|+01.27	Full: {'objectId': 'Watch|-01.45|+00

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/opt/conda/envs/pytorch-py3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/pytorch-py3.6/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/ai2thor/controller.py", line 697, in _start_unity_thread
    raise Exception("command: %s exited with %s" % (command, returncode))
Exception: command: ['/root/.ai2thor/releases/thor-201909061227-Linux64/thor-201909061227-Linux64', '-screen-fullscreen', '0', '-screen-quality', '4', '-screen-width', '300', '-screen-height', '300'] exited with -11



--KeyboardInterrupt--

Program interrupted. (Use 'cont' to resume).
Program interrupted. (Use 'cont' to resume).

In [12]:
# SCRIPT VERSION
# ('pick_two_obj_and_place', 'Watch', 'None', 'Dresser', 205)

!python scripts/sample_planner_trajectories.py --debug --goal pick_two_obj_and_place --pickup Watch --movable None --receptacle Dresser --scene 205 --seed 40 --save_path /root/data_alfred/demo_generated/

Found path: /root/.ai2thor/releases/thor-201909061227-Linux64/thor-201909061227-Linux64
Mono path[0] = '/root/.ai2thor/releases/thor-201909061227-Linux64/thor-201909061227-Linux64_Data/Managed'
Mono config path = '/root/.ai2thor/releases/thor-201909061227-Linux64/thor-201909061227-Linux64_Data/Mono/etc'
Preloaded 'ScreenSelector.so'
Display 0 'default': 1152x864 (primary device).
Logging to /root/.config/unity3d/Allen Institute for Artificial Intelligence/AI2-Thor/Player.log
ThorEnv started.
Force Unsave Success Data: False
Task: ('pick_two_obj_and_place', 'Watch', 'None', 'Dresser', '205')
PDDLGoalType: pick_two_obj_and_place
Saving images to: /root/data_alfred/demo_generated/new_trajectories/pick_two_obj_and_place-Watch-None-Dresser-205/trial_T20200611_235628_356382/raw_images/
Resetting ThorEnv
set seed in game_state_base reset 40
setup random goal ----------------------------------------------
seed None
info ('train', 6455)
----------------------------------------------------------

step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: RotateRight
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: RotateLeft
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: MoveAhead	Full: {'moveMagnitude': 0.25}
step action Action: RotateRight
step 

In [13]:
from utils.dataset_management_util import load_fails_from_disk